Lets`s read apache_logs  with known format. So, we can use regexp in separator

In [3]:
import re
import pandas as pd

data = pd.read_csv(
    'apache_logs',
    sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
    engine='python',
    na_values='-',
    header=None,
    usecols=[0, 3, 4, 5, 6, 7, 8],
    names=['ip', 'time', 'request', 'status', 'size', 'referer', 'user_agent'])

Got DataFrame

In [4]:
data.head()

,ip,time,request,status,size,referer,user_agent
0,83.149.9.216,[17/May/2015:10:05:03 +0000],"""GET /presentations/logstash-monitorama-2013/i...",200,203023.0,"""http://semicomplete.com/presentations/logstas...","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1..."
1,83.149.9.216,[17/May/2015:10:05:43 +0000],"""GET /presentations/logstash-monitorama-2013/i...",200,171717.0,"""http://semicomplete.com/presentations/logstas...","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1..."
2,83.149.9.216,[17/May/2015:10:05:47 +0000],"""GET /presentations/logstash-monitorama-2013/p...",200,26185.0,"""http://semicomplete.com/presentations/logstas...","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1..."
3,83.149.9.216,[17/May/2015:10:05:12 +0000],"""GET /presentations/logstash-monitorama-2013/p...",200,7697.0,"""http://semicomplete.com/presentations/logstas...","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1..."
4,83.149.9.216,[17/May/2015:10:05:07 +0000],"""GET /presentations/logstash-monitorama-2013/p...",200,2892.0,"""http://semicomplete.com/presentations/logstas...","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1..."


Filtering dataframe by column status with value = 404

In [13]:
counted = data.loc[data['status'] == '404']['ip'].value_counts()

And printing 10, 20 top lines of them

In [14]:
counted.head(10)

208.91.156.11     60
144.76.95.39      14
91.236.75.25       8
66.249.73.135      8
75.97.9.59         6
176.92.75.62       5
130.237.218.86     4
84.137.208.44      4
198.245.61.43      3
78.173.140.106     3
Name: ip, dtype: int64

In [15]:
counted.head(20)

208.91.156.11      60
144.76.95.39       14
91.236.75.25        8
66.249.73.135       8
75.97.9.59          6
176.92.75.62        5
130.237.218.86      4
84.137.208.44       4
198.245.61.43       3
78.173.140.106      3
188.165.243.45      3
95.78.54.93         3
195.250.34.144      3
111.199.235.239     2
144.76.194.187      2
122.166.142.108     2
193.244.33.47       2
89.107.177.18       2
204.62.56.3         2
212.90.148.107      2
Name: ip, dtype: int64